In [118]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [119]:
import pandas as pd
data = pd.read_csv('/content/drive/My Drive/sts-train.csv',error_bad_lines=False,delimiter='\t',header=None) 
data

b'Skipping line 2508: expected 7 fields, saw 8\nSkipping line 2649: expected 7 fields, saw 8\nSkipping line 2726: expected 7 fields, saw 8\nSkipping line 3070: expected 7 fields, saw 8\nSkipping line 3392: expected 7 fields, saw 8\nSkipping line 5515: expected 7 fields, saw 9\nSkipping line 5516: expected 7 fields, saw 9\nSkipping line 5517: expected 7 fields, saw 9\nSkipping line 5518: expected 7 fields, saw 9\nSkipping line 5519: expected 7 fields, saw 9\nSkipping line 5520: expected 7 fields, saw 9\nSkipping line 5521: expected 7 fields, saw 9\nSkipping line 5522: expected 7 fields, saw 9\nSkipping line 5523: expected 7 fields, saw 9\nSkipping line 5524: expected 7 fields, saw 9\nSkipping line 5525: expected 7 fields, saw 9\nSkipping line 5526: expected 7 fields, saw 9\nSkipping line 5527: expected 7 fields, saw 9\nSkipping line 5528: expected 7 fields, saw 9\nSkipping line 5529: expected 7 fields, saw 9\nSkipping line 5530: expected 7 fields, saw 9\nSkipping line 5531: expected 7 f

,0,1,2,3,4,5,6
0,main-captions,MSRvid,2012test,1,5.00,A plane is taking off.,An air plane is taking off.
1,main-captions,MSRvid,2012test,4,3.80,A man is playing a large flute.,A man is playing a flute.
2,main-captions,MSRvid,2012test,5,3.80,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...
3,main-captions,MSRvid,2012test,6,2.60,Three men are playing chess.,Two men are playing chess.
4,main-captions,MSRvid,2012test,9,4.25,A man is playing the cello.,A man seated is playing the cello.
...,...,...,...,...,...,...,...
5504,main-news,headlines,2015,1489,1.20,"Palestinian hunger striker, Israel reach deal",Palestinian activist detained in Israeli raid
5505,main-news,headlines,2015,1493,4.80,Assad says Syria will comply with UN arms reso...,Syria's Assad vows to comply with U.N. resolution
5506,main-news,headlines,2015,1496,4.60,South Korean President Sorry For Ferry Response,S. Korean president 'sorry' for ferry disaster
5507,main-news,headlines,2015,1498,0.00,Food price hikes raise concerns in Iran,American Chris Horner wins Tour of Spain


In [120]:
dtest = pd.read_csv('/content/drive/My Drive/sts-dev.csv',error_bad_lines=False,delimiter='\t',header=None) 
dtest

b'Skipping line 1041: expected 7 fields, saw 8\nSkipping line 1065: expected 7 fields, saw 8\nSkipping line 1082: expected 7 fields, saw 8\nSkipping line 1136: expected 7 fields, saw 8\nSkipping line 1149: expected 7 fields, saw 8\nSkipping line 1449: expected 7 fields, saw 9\nSkipping line 1450: expected 7 fields, saw 9\nSkipping line 1451: expected 7 fields, saw 9\nSkipping line 1452: expected 7 fields, saw 9\nSkipping line 1453: expected 7 fields, saw 9\nSkipping line 1454: expected 7 fields, saw 9\nSkipping line 1455: expected 7 fields, saw 9\nSkipping line 1456: expected 7 fields, saw 9\nSkipping line 1457: expected 7 fields, saw 9\nSkipping line 1458: expected 7 fields, saw 9\nSkipping line 1459: expected 7 fields, saw 9\nSkipping line 1460: expected 7 fields, saw 9\nSkipping line 1461: expected 7 fields, saw 9\nSkipping line 1462: expected 7 fields, saw 9\nSkipping line 1463: expected 7 fields, saw 9\nSkipping line 1464: expected 7 fields, saw 9\nSkipping line 1465: expected 7 f

,0,1,2,3,4,5,6
0,main-captions,MSRvid,2012test,0,5.00,A man with a hard hat is dancing.,A man wearing a hard hat is dancing.
1,main-captions,MSRvid,2012test,2,4.75,A young child is riding a horse.,A child is riding a horse.
2,main-captions,MSRvid,2012test,3,5.00,A man is feeding a mouse to a snake.,The man is feeding a mouse to the snake.
3,main-captions,MSRvid,2012test,7,2.40,A woman is playing the guitar.,A man is playing guitar.
4,main-captions,MSRvid,2012test,8,2.75,A woman is playing the flute.,A man is playing a flute.
...,...,...,...,...,...,...,...
1438,main-news,headlines,2015,1417,4.60,World's oldest man dies at 116,Japan: World's Oldest Man Ever Dies Aged 116
1439,main-news,headlines,2015,1426,3.40,Pakistanis vote in landmark election,Pakistan holds landmark election as 29 killed ...
1440,main-news,headlines,2015,1447,0.40,Algeria president gets therapy after stroke: s...,Bulgarian president tries to break election st...
1441,main-news,headlines,2015,1471,0.00,Yemen: Fighting in north kills over 120,Janet Yellen: Five things you might not know


In [121]:
!pip install pytorch-pretrained-bert

In [0]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [0]:
#Two sentence input form: [CLS] The man went to the store. [SEP] He bought a gallon of milk. [SEP]
data[5]="[CLS]"+data[5]+"[SEP]"
data[6]="[CLS]"+data[6]+"[SEP]"

In [124]:
model=BertModel.from_pretrained('bert-base-uncased')
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Lin

In [0]:
def getEmbedding(i,j,dt):
  s1=tokenizer.tokenize(dt.iloc[i,j])
  #mapping to vocabulary
  inds1=tokenizer.convert_tokens_to_ids(s1)
  segid=[0]*len(s1)
  ttensor=torch.tensor([inds1])
  stensor=torch.tensor([segid])
  with torch.no_grad():
    layers,_=model(ttensor,stensor)
  #print (type(layers))
  tnembed=torch.stack(layers,dim=0) #list to tensors
  tnembed=torch.squeeze(tnembed,dim=1) #removing batch dimension
  tnembed=tnembed.permute(1,0,2) #changing to words,layers,features


  #print (tnembed.size())
  embedding=torch.mean(tnembed,dim=0)
  embedding=torch.mean(embedding,dim=0)
  #vec=tnembed[:,11,:]
  #print (vec.size())
  #embedding=torch.mean(vec,dim=0)
  #print (embedding.size())
  return embedding

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
def getScore(i,dt):
  e1=getEmbedding(i,5,dt)
  e2=getEmbedding(i,6,dt)
  em1=np.array(e1)
  em2=np.array(e2)
  embed1=np.array([em1])
  embed2=np.array([em2])
  score = cosine_similarity(embed1,embed2)
  #score = -tf.acos(tf.reduce_sum(tf.multiply(e1, e2), axis=1))
  return score

In [0]:
sim=np.array([0.0]*dtest.shape[0],dtype=float)
for i in range(dtest.shape[0]):
  sim[i]=getScore(i,data)
  #print (i)

In [0]:
import scipy
from scipy.stats import pearsonr
label=np.array(dtest.iloc[:,4],dtype=float)
pearson_correlation = scipy.stats.pearsonr(sim, label)

In [138]:
print (pearson_correlation[0])
print (pearson_correlation[1])

-0.007173023645250704
0.7854314349786126


In [0]:
#print (tnembed.size())
  #embedding=torch.mean(tnembed,dim=0)
  #embedding=torch.mean(embedding,dim=0)
  tembedding=[]
  for t in tnembed:
    vec=torch.cat((t[-1],t[-2],t[-3],t[-4]),dim=0)
    tembedding.append(vec)
  temp=torch.stack(tembedding,dim=0)
  #embedding=tf.reshape(tembedding,[-1])
  print (temp.size())
  #print (embedding.size())
  #embedding=tnembed